In [1]:
import math
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import lightgbm
from lightgbm import LGBMRegressor

import pickle

In [2]:
train_set = pd.read_csv('./output/preprocessed_train.csv')
train_set

,segment_id,sensor_1_sum,sensor_1_mean,sensor_1_std,sensor_1_var,sensor_1_max,sensor_1_min,sensor_1_skew,sensor_1_mad,sensor_1_kurtosis,...,missed_percent_sensor1,missed_percent_sensor2,missed_percent_sensor3,missed_percent_sensor4,missed_percent_sensor5,missed_percent_sensor6,missed_percent_sensor7,missed_percent_sensor8,missed_percent_sensor9,missed_percent_sensor10
0,1136037770,-96621.0,-1.610323,303.096099,91867.245099,1353.0,-1421.0,-0.057494,199.0,0.247736,...,0.0,0.0,0.0,0.0,1.000,0.0,0.0,0.0,0.000000,0.000000
1,1969647810,85569.0,1.426126,438.360560,192159.980413,2197.0,-2005.0,-0.001189,268.0,1.177624,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.012300,0.000000
2,1895879680,150278.0,2.504592,241.575415,58358.681050,1148.0,-1115.0,0.071632,157.0,0.567813,...,0.0,0.0,0.0,0.0,1.000,0.0,0.0,0.0,0.000000,0.000000
3,2068207140,129950.0,2.165797,221.967825,49269.715377,1254.0,-1025.0,0.123639,138.0,1.079660,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.042066,0.000000
4,192955606,4429.0,0.073815,261.695935,68484.762635,1114.0,-1190.0,0.006594,171.0,0.274037,...,0.0,1.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4426,873340274,54405.0,0.906735,613.427301,376293.054135,2916.0,-2610.0,-0.003413,412.0,0.168746,...,0.0,0.0,0.0,0.0,0.012,0.0,0.0,0.0,0.000000,0.000000
4427,1297437712,476221.0,7.936884,649.089618,421317.332733,4498.0,-3654.0,0.113867,422.0,0.695682,...,0.0,1.0,0.0,0.0,1.000,0.0,0.0,0.0,0.010600,0.000000
4428,694853998,85261.0,1.420993,110.942596,12308.259562,557.0,-753.0,-0.060527,70.0,1.005580,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000000,0.000000
4429,1886987043,54350.0,0.905818,478.852581,229299.794013,3938.0,-3555.0,0.028062,282.0,2.278329,...,0.0,1.0,0.0,0.0,0.000,0.0,0.0,1.0,0.000000,0.068466


In [3]:
drop_cols = list()
for col in train_set.columns:
    if col == 'segment_id':
        continue
    if abs(train_set[col].corr(train_set['time_to_eruption'])) < 0.01:
        drop_cols.append(col)

In [4]:
not_to_drop_cols = list()

for col1 in train_set.columns:
    for col2 in train_set.columns:
        if col1 == col2:
            continue
        if col1 == 'segment_id' or col2 == 'segment_id': 
            continue
        if col1 == 'time_to_eruption' or col2 == 'time_to_eruption':
            continue
        if abs(train_set[col1].corr(train_set[col2])) > 0.98:
            if col2 not in drop_cols and col1 not in not_to_drop_cols:
                drop_cols.append(col2)
                not_to_drop_cols.append(col1)

In [5]:
train = train_set.drop(['segment_id', 'time_to_eruption'], axis=1)
y = train_set['time_to_eruption']

In [6]:
reduced_y = y.copy()
reduced_train = train.copy()
reduced_train = reduced_train.drop(drop_cols, axis=1)
reduced_train

,sensor_1_var,sensor_1_skew,sensor_1_kurtosis,sensor_1_fft_real_max,sensor_1_fft_real_min,sensor_2_var,sensor_2_max,sensor_2_min,sensor_2_skew,sensor_2_kurtosis,...,sensor_10_fft_real_max,sensor_10_fft_real_min,has_missed_sensors,missed_percent_sensor1,missed_percent_sensor2,missed_percent_sensor3,missed_percent_sensor5,missed_percent_sensor6,missed_percent_sensor7,missed_percent_sensor8
0,91867.245099,-0.057494,0.247736,1.531152e+06,-2.313401e+06,460132.712015,5572.0,-5199.0,-0.003121,2.512873,...,3.722646e+06,-5.142345e+06,1,0.0,0.0,0.0,1.000,0.0,0.0,0.0
1,192159.980413,-0.001189,1.177624,1.408965e+06,-1.523921e+06,437136.850129,5111.0,-4267.0,0.033019,2.028709,...,4.095983e+06,-2.306340e+06,0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
2,58358.681050,0.071632,0.567813,1.794606e+06,-9.249402e+05,691341.065208,7587.0,-6147.0,0.143010,4.840094,...,1.398204e+06,-2.886983e+06,1,0.0,0.0,0.0,1.000,0.0,0.0,0.0
3,49269.715377,0.123639,1.079660,8.590981e+05,-7.839216e+05,180786.191204,2453.0,-2828.0,-0.013416,1.776726,...,2.441209e+06,-2.359133e+06,0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
4,68484.762635,0.006594,0.274037,8.162634e+05,-1.031017e+06,0.000000,0.0,0.0,0.000000,0.000000,...,1.360352e+06,-1.527639e+06,1,0.0,1.0,0.0,0.000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4426,376293.054135,-0.003413,0.168746,2.516297e+06,-3.617968e+06,885004.123726,6661.0,-5989.0,0.004394,1.996367,...,4.707731e+06,-3.847390e+06,0,0.0,0.0,0.0,0.012,0.0,0.0,0.0
4427,421317.332733,0.113867,0.695682,1.720791e+06,-1.949110e+06,0.000000,0.0,0.0,0.000000,0.000000,...,2.732602e+06,-3.027802e+06,1,0.0,1.0,0.0,1.000,0.0,0.0,0.0
4428,12308.259562,-0.060527,1.005580,5.927983e+05,-6.477082e+05,255755.237717,4169.0,-4663.0,-0.058501,5.235877,...,5.966443e+05,-6.632181e+05,0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
4429,229299.794013,0.028062,2.278329,1.005809e+06,-9.846973e+05,0.000000,0.0,0.0,0.000000,0.000000,...,1.590214e+06,-1.448093e+06,1,0.0,1.0,0.0,0.000,0.0,0.0,1.0


In [7]:
train, val, y, y_val = train_test_split(train, y, random_state=42, test_size=0.2, shuffle=True)
reduced_train, reduced_val, reduced_y, reduced_y_val = train_test_split(reduced_train, reduced_y, random_state=42, test_size=0.2, shuffle=True)

In [8]:
lgb = LGBMRegressor(
    random_state=42, 
    max_depth=7, 
    n_estimators=250, 
    learning_rate=0.12
)

lgb.fit(train, y)
preds = lgb.predict(val)

C:\Users\lxsbx\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:110: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\lxsbx\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 217, in _count_physical_cores
    raise ValueError(


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010052 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 57900
[LightGBM] [Info] Number of data points in the train set: 3544, number of used features: 240
[LightGBM] [Info] Start training from score 22780690.250564
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [War

In [9]:
def rmse(y_true, y_pred):
    return math.sqrt(mse(y_true, y_pred))

In [10]:
print('Simple LGB model rmse: ', rmse(y_val, preds))

Simple LGB model rmse:  5882715.716720319


In [11]:
test = pd.read_csv('./output/preprocessed_test.csv')
test

,sensor_1_sum,sensor_1_mean,sensor_1_std,sensor_1_var,sensor_1_max,sensor_1_min,sensor_1_skew,sensor_1_mad,sensor_1_kurtosis,sensor_1_quantile99,...,missed_percent_sensor1,missed_percent_sensor2,missed_percent_sensor3,missed_percent_sensor4,missed_percent_sensor5,missed_percent_sensor6,missed_percent_sensor7,missed_percent_sensor8,missed_percent_sensor9,missed_percent_sensor10
0,267753.0,4.462476,306.174474,9.374281e+04,1347.0,-1319.0,0.018572,198.0,0.441816,758.0,...,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
1,50287.0,0.838103,335.591100,1.126214e+05,1214.0,-1268.0,-0.011491,231.0,-0.097694,768.0,...,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
2,47695.0,0.794903,352.172248,1.240253e+05,4259.0,-3806.0,-0.050056,193.0,6.125681,927.0,...,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.000000,0.000000
3,117225.0,1.953717,259.030122,6.709660e+04,1220.0,-1156.0,0.014705,175.0,0.083266,609.0,...,0.0,1.0,0.0,0.0,0.274645,0.0,0.0,0.0,0.000000,0.004867
4,-239079.0,-3.984584,254.306426,6.467176e+04,972.0,-1111.0,-0.077960,170.0,0.058601,577.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4515,-8140616.0,-135.674672,29486.243321,8.694385e+08,32767.0,-32767.0,0.000677,32767.0,-1.822876,32767.0,...,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,1.000000,1.000000
4516,-159989.0,-2.666439,294.261065,8.658957e+04,1032.0,-1291.0,-0.046301,196.0,0.060913,676.0,...,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.002400,0.000000
4517,71538.0,1.192280,371.072752,1.376950e+05,2641.0,-3032.0,0.058241,226.0,1.870325,919.0,...,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,1.000000,1.000000
4518,193117.0,3.218563,346.656999,1.201711e+05,1494.0,-1423.0,0.084854,223.0,0.421511,875.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.049766,0.000000


In [12]:
reduced_test = test.copy()
reduced_test = reduced_test.drop(drop_cols, axis=1)
reduced_test

,sensor_1_var,sensor_1_skew,sensor_1_kurtosis,sensor_1_fft_real_max,sensor_1_fft_real_min,sensor_2_var,sensor_2_max,sensor_2_min,sensor_2_skew,sensor_2_kurtosis,...,sensor_10_fft_real_max,sensor_10_fft_real_min,has_missed_sensors,missed_percent_sensor1,missed_percent_sensor2,missed_percent_sensor3,missed_percent_sensor5,missed_percent_sensor6,missed_percent_sensor7,missed_percent_sensor8
0,9.374281e+04,0.018572,0.441816,9.405932e+05,-1.193852e+06,0.000000e+00,0.0,0.0,0.000000,0.000000,...,-0.000000e+00,-0.000000e+00,1,0.0,1.0,0.0,0.000000,0.0,0.0,0.0
1,1.126214e+05,-0.011491,-0.097694,2.409332e+06,-1.739455e+06,0.000000e+00,0.0,0.0,0.000000,0.000000,...,2.450837e+06,-2.194674e+06,1,0.0,1.0,0.0,0.000000,0.0,0.0,0.0
2,1.240253e+05,-0.050056,6.125681,7.435363e+05,-9.281370e+05,2.564741e+05,3140.0,-3552.0,-0.079326,1.494840,...,-0.000000e+00,-0.000000e+00,1,0.0,0.0,0.0,1.000000,0.0,0.0,0.0
3,6.709660e+04,0.014705,0.083266,7.332782e+05,-8.965958e+05,0.000000e+00,0.0,0.0,0.000000,0.000000,...,-0.000000e+00,-0.000000e+00,1,0.0,1.0,0.0,0.274645,0.0,0.0,0.0
4,6.467176e+04,-0.077960,0.058601,1.108086e+06,-9.995154e+05,5.569712e+05,11615.0,-10521.0,0.086848,11.155720,...,1.338345e+06,-1.480067e+06,0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4515,8.694385e+08,0.000677,-1.822876,4.117357e+08,-2.948841e+08,3.600710e+08,32767.0,-32767.0,0.059092,-0.833496,...,-0.000000e+00,-0.000000e+00,1,0.0,0.0,0.0,1.000000,0.0,0.0,0.0
4516,8.658957e+04,-0.046301,0.060913,1.371946e+06,-1.565973e+06,6.163623e+05,7872.0,-6508.0,-0.032813,6.198089,...,3.571613e+06,-2.314506e+06,1,0.0,0.0,0.0,1.000000,0.0,0.0,0.0
4517,1.376950e+05,0.058241,1.870325,2.003819e+06,-2.706902e+06,7.322297e+04,1719.0,-1848.0,0.021956,1.116785,...,-0.000000e+00,-0.000000e+00,1,0.0,0.0,0.0,1.000000,0.0,0.0,0.0
4518,1.201711e+05,0.084854,0.421511,1.399447e+06,-1.789332e+06,3.284338e+05,5652.0,-3670.0,0.093348,2.408287,...,3.465824e+06,-3.755317e+06,0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [13]:
lgb.booster_.save_model('./output/lgb_model.txt')

In [14]:
lgb = lightgbm.Booster(model_file='./output/lgb_model.txt')

In [15]:
preds1 = lgb.predict(test)
preds1

array([22078765.52459123, 38785017.54312659, 33192544.9307948 , ...,
       26984081.18293213, 22917329.55271703, 26860919.21262011])